# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [9]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [8]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [11]:
w = data[data.race=='w']
b = data[data.race=='b']
#1. Z test since sample size larger than 30 based on the CLT
#2.null hypothesis: Race doesnt have a significant impact on the rate of callbacks
#alternate hypothesis: Race does have a significant impact on the rate of callbacks

In [12]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [30]:
total_w = w.race.count()
p_w = sum(w[w.race=='w'].call)/w.race.count()
q_w = 1- p_w
#fraction of white sounding names who got a call

In [33]:
total_b = b.race.count()
p_b = sum(b[b.race=='b'].call)/b.race.count()
q_b = 1 - p_b
#fraction of black sounding names who got a call

In [48]:
p_hat = (total_w * p_w + total_b * p_b) / (total_b + total_w)
p_hat

0.08049281314168377

In [60]:
z_val = (p_w - p_b) / np.sqrt(p_hat * (1 - p_hat) * (1/total_w + 1/total_b))
z_critical = stats.norm.ppf(q = 0.975)
z_val

4.108412152434346

In [50]:
print(z_critical)
print(p_w)
print(q_w)
print(p_b)
print(q_b)

1.959963984540054
0.09650924024640657
0.9034907597535934
0.06447638603696099
0.935523613963039


In [56]:
#margin of error and confidence interval of .95
upper_interval = (p_w - p_b) + 1.96 * np.sqrt((.096509*.903490/2435) + (.06447*.9355/2435))
lower_interval = (p_w - p_b) - 1.96 * np.sqrt((.096509*.903490/2435) + (.06447*.9355/2435))

In [57]:
print(upper_interval)
print(lower_interval)

0.04728785789897039
0.016777850519920787


In [58]:
import scipy
scipy.stats.norm.sf(abs(z_val))*2 

3.983886837585077e-05

In [ ]:
#result significant reject null hyp

In [61]:
#Start of a two sample bootstrap test
#define the function 
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

In [62]:
def draw_bs_reps(data, func, size):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [88]:
mean_call = (p_w + p_b) / 2

In [89]:
mean_call

0.08049281314168377

In [90]:
p_w_shifted = w.call - np.mean(w.call) + mean_call
p_b_shifted = b.call - np.mean(b.call) + mean_call

In [95]:
# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_a = draw_bs_reps(p_w_shifted, np.mean, size=10000)
bs_replicates_b = draw_bs_reps(p_b_shifted, np.mean, size=10000)

In [96]:
np.mean(w.call)

0.09650924056768417

In [97]:
bs_replicates_a

array([0.07474332, 0.07802874, 0.08008213, ..., 0.07145791, 0.07802874,
       0.07802874])

In [98]:
bs_replicates_b

array([0.07597538, 0.07967148, 0.08829571, ..., 0.0792608 , 0.0792608 ,
       0.08459961])

In [99]:
empirical_diff_means = p_w - p_b

In [105]:
bs_replicates = bs_replicates_a - bs_replicates_b

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)

p-value = 0.0001


In [107]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
Ans: The researchers took great care to make sure that the resumes were identical aside from the names. So, examining the resumes more closely for differences in education, work experience, etc. should not be helpful if the study was executed properly.
The methods of the study could be examined more closely for sample bias in the cities and the hiring managers selected, etc.